# Registro de actividades formativas

Base de datos para hacer funcionar un sistema de registro/aprovacion/validación de actividades formativas para los alumnos UG

## Creación de la base de datos

In [8]:
CREATE DATABASE UGTO
GO

: Msg 1801, Level 16, State 1, Line 1
Database 'UGTO' already exists. Choose a different database name.

Total execution time: 00:00:10.217

## Definición de tablas

In [9]:
use UGTO
GO

CREATE TABLE Personas(
    idPersona int not null identity primary key,
    Nombre varchar(25) not null,
    ApellidoP VARCHAR(25) not null,
    ApellidoM varchar(25) not  null,
    Edad int,
    Sexo varchar(1) not null, -- M masculino, F = Femenino, O = No binario
    RFC varchar(12) not null,
    Correo varchar(50) not null,
    Telefono varchar(10) not null,
    CURP varchar(18) not null);

CREATE TABLE Carrera(
    idCarrera int not null identity primary key,
    Nombre varchar(50) not null);

CREATE TABLE Alumnos(
    NUA varchar(6) primary key,
    idPersona int,
    idCarrera int,
    Semestre varchar(2) not null,
    Promedio varchar(5) not null,
    foreign key(idPersona) REFERENCES Personas (idPersona),
    foreign key(idCarrera) REFERENCES Carrera (idCarrera));

create table Administrativos(
    NUE varchar(6) not null primary key, 
	idPersona int,
	Foreign key(idPersona) References Personas (idPersona),
	pendientes int default 0,
	mensajes varchar (250));

create table estado(
    idStatus int not null identity primary key,
    Descripcion varchar (25));

CREATE table Subarea(
	idSubarea int not null IDENTITY PRIMARY KEY,
	nombreArea varchar (35));

CREATE table actividad (
    idActividad int not null identity primary key,
   	titulo varchar (200) not null,
    descripcion varchar (500) not null,
    idSubarea int,
    Foreign key(idSubarea) References Subarea (idSubarea),
    idCarrera int,
    Foreign key(idCarrera) References Carrera (idCarrera),
    NUA varchar(6),
    Foreign key(NUA) References Alumnos (NUA),  
    fechaInicio date  not null,
    fechaFin date not null,
    noHoras int not null,
    organizador varchar (100),
    evidencia varchar(max),
    idStatus int DEFAULT 0, 
    Foreign key(idStatus) References estado (idStatus));

CREATE TABLE avance (
    idavance int not null IDENTITY PRIMARY KEY,
    NUA varchar(6),
    Foreign key(NUA) References Alumnos (NUA),	
    idSubarea int,
    Foreign key(idSubarea) References Subarea (idSubarea),
    creditos decimal (5,2) NOT null DEFAULT 0,
    creditosFaltan decimal (5,2) not null,
    noActividades int not null DEFAULT 0,
    porcentaje int not null DEFAULT 0);

: Msg 40508, Level 16, State 1, Line 1
USE statement is not supported to switch between databases. Use a new connection to connect to a different database.

: Msg 229, Level 14, State 5, Line 20
The REFERENCES permission was denied on the object 'Personas', database 'master', schema 'dbo'.

: Msg 1088, Level 16, State 20, Line 20
Cannot find the object "Personas" because it does not exist or you do not have permissions.

: Msg 1750, Level 16, State 1, Line 20
Could not create constraint or index. See previous errors.

Total execution time: 00:00:00.128

In [ ]:

CREATE PROCEDURE sp_insertAlumnos(
    @Nombre varchar(25),
    @ApellidoP VARCHAR(25),
    @ApellidoM varchar(25),
    @Edad int,
    @Sexo varchar(1),
    @RFC varchar(12),
    @Correo varchar(50),
    @Telefono varchar(50),
    @CURP varchar(18),
    @NUA varchar (6),
    @idCarrera int,
    @Semestre varchar(2),
    @Promedio varchar(5) )
    AS
    BEGIN
    SET NOCOUNT ON;
        DECLARE @idPersona int;
        INSERT into Personas (Nombre, ApellidoP, ApellidoM, Edad, Sexo, RFC, Correo, Telefono, CURP)
        VALUES (UPPER(@Nombre), UPPER(@ApellidoP), UPPER(@ApellidoM), @Edad, @Sexo, UPPER(@RFC), @Correo, @Telefono, UPPER(@CURP));

    	SET @idPersona = SCOPE_IDENTITY();

        INSERT into Alumnos (NUA, idPersona, idCarrera, Semestre, Promedio)
        VALUES (@NUA, @idPersona, @idCarrera, @Semestre, @Promedio);
END
go


CREATE PROCEDURE sp_insertAdmin(
    @Nombre varchar(25),
    @ApellidoP VARCHAR(25),
    @ApellidoM varchar(25),
    @Edad int,
    @Sexo varchar(1),
    @RFC varchar(12),
    @Correo varchar(50),
    @Telefono varchar(50),
    @CURP varchar(18),
    @NUE varchar(6) )
    AS
    BEGIN
    SET NOCOUNT ON;
        DECLARE @idPersona int;
        INSERT into Personas (Nombre, ApellidoP, ApellidoM, Edad, Sexo, RFC, Correo, Telefono, CURP)
        VALUES (UPPER(@Nombre), UPPER(@ApellidoP), UPPER(@ApellidoM), @Edad, @Sexo, UPPER(@RFC), @Correo, @Telefono, UPPER(@CURP));

    	SET @idPersona = SCOPE_IDENTITY();

        INSERT into Administrativos (NUE, idPersona )
        VALUES (@NUE, @idPersona);
END
go


drop PROCEDURE sp_cambiarStatus
go
CREATE PROCEDURE sp_cambiarStatus(
    @idAct int,
    @nuevoStatus int,
	@nuevasHoras int)
    AS
    BEGIN
    SET NOCOUNT ON;
		if @nuevoStatus != 3 begin
        	UPDATE actividad 
        	SET idStatus = @nuevoStatus
    		WHERE idActividad = @idAct;
			end
		else begin
			UPDATE actividad 
        	SET idStatus = @nuevoStatus, noHoras = @nuevasHoras
    		WHERE idActividad = @idAct;
			end
END
go


CREATE PROCEDURE sp_insertActividades(
    @titulo varchar (200),
    @descripcion varchar (500),
    @idSubarea int,
    @idCarrera int,
    @NUA varchar (6),
    @fechaInicio date,
    @fechaFin date,
    @noHoras int,
    @organizador varchar (100),
    @evidencia varchar(max))
    AS
    BEGIN
    SET NOCOUNT ON;
        INSERT into actividad (titulo, descripcion,idSubarea, idCarrera, NUA, fechaInicio, fechaFin, noHoras, organizador, evidencia, idStatus)
        VALUES(UPPER(@titulo), UPPER(@descripcion),@idSubarea, @idCarrera, @NUA, @fechaInicio, @fechaFin, @noHoras, UPPER(@organizador), @evidencia, 1);
END

In [ ]:
INSERT INTO Carrera (Nombre) VALUES ('Ingeniería en comunicaciones y electrónica');
INSERT INTO Carrera (Nombre) VALUES ('Ingeniería en sistemas computacionales');
INSERT INTO Carrera (Nombre) VALUES ('Gestión empresarial');
INSERT INTO Carrera (Nombre) VALUES ('Enseñanza del inglés');

INSERT INTO Subarea (nombreArea) VALUES ('Desarrollo personal');
INSERT INTO Subarea (nombreArea) VALUES ('Responsabilidad social');
INSERT INTO Subarea (nombreArea) VALUES ('Creatividad y espíritu emprendedor');
INSERT INTO Subarea (nombreArea) VALUES ('Formación cultural e intercultural');
INSERT INTO Subarea (nombreArea) VALUES ('Área complementaria');


INSERT INTO estado (Descripcion) VALUES ('Enviado');
INSERT INTO estado (Descripcion) VALUES ('Aceptado');
INSERT INTO estado (Descripcion) VALUES ('Modificado');
INSERT INTO estado (Descripcion) VALUES ('Rechazado');

In [ ]:
CREATE FUNCTION calcularCredFal (@car int, @sub int)
    RETURNS decimal (5,2)
    BEGIN
	DECLARE @pe varchar(50); 
    DECLARE @area varchar(50); 
	DECLARE @creditos DECIMAL(5,2);
    SELECT @pe = Nombre FROM Carrera WHERE idCarrera = @car;
    SELECT @area = nombreArea FROM Subarea WHERE idSubarea = @sub;
   
    IF @pe = 'Enseñanza del inglés' BEGIN
        IF @area = 'Desarrollo personal' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Responsabilidad social' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Creatividad y espíritu emprendedor' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Formación cultural e intercultural' BEGIN
        	SET @creditos = 10;
        END
        ELSE IF @area = 'Área complementaria' BEGIN
        	SET @creditos = 0;
        END
        ELSE BEGIN
        	SET @creditos = 0;
        END 
    END
    ELSE BEGIN
    	IF @area = 'Desarrollo personal' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Responsabilidad social' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Creatividad y espíritu emprendedor' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Formación cultural e intercultural' BEGIN
        	SET @creditos = 2;
        END
        ELSE IF @area = 'Área complementaria' BEGIN
        	SET @creditos = 5;
        END
        ELSE BEGIN
        	SET @creditos = 0;
        END 
    END 
	
	RETURN @creditos;

END
GO



CREATE FUNCTION calcularPorcentajeAvance (@creditos decimal (5,2), @creditosTotales decimal (5,2))
    RETURNS decimal(5,2)
    BEGIN
    DECLARE @porcentaje decimal(5,2);
    SET @porcentaje = (@creditos / @creditosTotales) * 100;
    RETURN @porcentaje;
END
GO


CREATE FUNCTION horascredito (@hours int)
    RETURNS decimal(5,2)
    BEGIN
    DECLARE @creditos decimal(5,2);
    SET @creditos = (1.0/25)*@hours;
    RETURN @creditos;
END
GO


In [ ]:
CREATE TRIGGER TR_InsertAvance ON Alumnos
AFTER INSERT AS
BEGIN
    SET NOCOUNT ON;
    DECLARE @idCarrera_temp INT;
    SELECT @idCarrera_temp = idCarrera FROM inserted;

    INSERT INTO avance (NUA, idSubarea, creditosFaltan)
    VALUES ((SELECT NUA FROM inserted), 1, dbo.calcularCredFal(@idCarrera_temp, 1));

    INSERT INTO avance (NUA, idSubarea, creditosFaltan)
    VALUES ((SELECT NUA FROM inserted), 2, dbo.calcularCredFal(@idCarrera_temp, 2));

    INSERT INTO avance (NUA, idSubarea, creditosFaltan)
    VALUES ((SELECT NUA FROM inserted), 3, dbo.calcularCredFal(@idCarrera_temp, 3));

    INSERT INTO avance (NUA, idSubarea, creditosFaltan)
    VALUES ((SELECT NUA FROM inserted), 4, dbo.calcularCredFal(@idCarrera_temp, 4));

    INSERT INTO avance (NUA, idSubarea, creditosFaltan)
    VALUES ((SELECT NUA FROM inserted), 5, dbo.calcularCredFal(@idCarrera_temp, 5));
END
GO

CREATE TRIGGER TR_updateAvance ON actividad
AFTER UPDATE AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @nuevaCantidad DECIMAL(5,2);
    SET @nuevaCantidad = dbo.horascredito((SELECT noHoras FROM INSERTED));

    IF (SELECT idStatus FROM INSERTED) = 2 OR (SELECT idStatus FROM INSERTED) = 3
    BEGIN
        UPDATE avance
        SET creditos = creditos + @nuevaCantidad,
            creditosFaltan = creditosFaltan - @nuevaCantidad,
            noActividades = noActividades + 1,
            porcentaje = dbo.calcularPorcentajeAvance( creditos, dbo.calcularCredFal((SELECT idCarrera FROM INSERTED),(SELECT idSubarea FROM INSERTED)))
        WHERE NUA = (SELECT NUA FROM INSERTED)
        AND idSubarea = (SELECT idSubarea FROM INSERTED);
    END
    ELSE
    BEGIN
        UPDATE alumnos
        SET mensaje = 'Actividad rechazada'
        WHERE NUA = (SELECT NUA FROM INSERTED);
    END;

    UPDATE administrativos
    SET pendientes = pendientes - 1,
        mensajes = CONCAT('Usted tiene ', pendientes -1 , ' por revisar');
END
GO

CREATE TRIGGER TR_mensajeAlumno ON actividad
AFTER UPDATE AS
BEGIN
    IF UPDATE(idStatus)
    BEGIN
        IF (SELECT idStatus FROM inserted) = 1
        BEGIN
            UPDATE alumnos
            SET mensaje = CONCAT('La actividad con ID: ', (SELECT idActividad FROM inserted), ' ha sido enviada')
            WHERE NUA = (SELECT NUA FROM inserted);
        END
        ELSE IF (SELECT idStatus FROM inserted) = 2
        BEGIN
            UPDATE alumnos
            SET mensaje = CONCAT('La actividad con ID: ', (SELECT idActividad FROM inserted), ' ha sido aceptada')
            WHERE NUA = (SELECT NUA FROM inserted);
        END
        ELSE IF (SELECT idStatus FROM inserted) = 3
        BEGIN
            UPDATE alumnos
            SET mensaje = CONCAT('La actividad con ID: ', (SELECT idActividad FROM inserted), ' ha sido modificada y aceptada')
            WHERE NUA = (SELECT NUA FROM inserted);
        END
        ELSE IF (SELECT idStatus FROM inserted) = 4
        BEGIN
            UPDATE alumnos
            SET mensaje = CONCAT('La actividad con ID: ', (SELECT idActividad FROM inserted), ' ha sido rechazada')
            WHERE NUA = (SELECT NUA FROM inserted);
        END
    END
END
go


CREATE TRIGGER TR_mensajeAdmin ON actividad
AFTER INSERT AS
BEGIN
    SET NOCOUNT ON;

    UPDATE administrativos
    SET pendientes = pendientes + 1,
        mensajes = CONCAT('Usted tiene ', pendientes +1 , ' por revisar');
END
go

In [ ]:
CREATE VIEW listaPendientes AS 
SELECT personas.Nombre, alumnos.NUA, actividad.titulo 
FROM personas
INNER JOIN alumnos ON personas.idPersona = alumnos.idPersona
INNER JOIN actividad ON alumnos.NUA = actividad.NUA
WHERE actividad.idStatus = 1
go


CREATE VIEW revisarActividad AS 
SELECT p.Nombre as "Nombre",  al.NUA, c.Nombre as "Carrera", s.nombreArea as "Area",  a.titulo as "Titulo", a.descripcion as "descripción", a.fechaInicio as "Inicio", a.fechaFin as "Fin", a.noHoras as "Horas", a.organizador, a.evidencia
FROM personas as p
INNER JOIN alumnos as al ON p.idPersona = al.idPersona
INNER JOIN Carrera as c ON al.idCarrera= c.idCarrera
LEFT JOIN actividad as a ON al.NUA = a.NUA
INNER JOIN Subarea as s ON al.idCarrera= c.idCarrera
WHERE a.idStatus = 1
go

CREATE VIEW listaAlumnos AS 
SELECT personas.Nombre, alumnos.NUA, alumnos.idCarrera
FROM personas
INNER JOIN alumnos ON personas.idPersona = alumnos.idPersona
go

CREATE VIEW bienvenida AS 
SELECT 
    CONCAT('Bienvenida de vuelta ', a.Nombre, ' ', a.ApellidoP, ' ',  a.ApellidoM, ' ', ad.mensajes) AS MensajeBienvenida
FROM 
    [dbo].[Personas] a
JOIN 
    [dbo].[Administrativos] ad ON a.idPersona = ad.idPersona;

CREATE VIEW verAvance AS 
select p.Nombre, 
	p.ApellidoP, 
	a.NUA, 
	a.idCarrera, 
	av.idSubarea as Area, 
	av.creditos as Creditos, 
	av.creditosFaltan as 'Creditos faltantes', 
	av.noActividades as 'No. actividaes', 
	Concat (av.porcentaje, '%') as 'Porcentaje de avance'
from [dbo].[Personas] p 
inner join [dbo].[Alumnos] a on a.idPersona = p.idPersona
inner join [dbo].[avance] av on av.NUA = a.NUA;

In [ ]:
EXEC sp_insertAdmin @Nombre = 'Ana Clara',
                   @ApellidoP = 'Montero',
                   @ApellidoM = 'Moreno',
                   @Edad = '35',
                   @Sexo = 'F',
                   @RFC = '1234567890RFC',
                   @Correo = 'e_sirangua@id.uff.br',
                   @Telefono = '4451814099',
                   @CURP = 'AAAA010101HGTAAA1',
                   @NUE = '000000'
go

EXEC  sp_insertAlumnos @Nombre = 'Emmanuel',
                   @ApellidoP = 'Torres',
                   @ApellidoM = 'Sirangua',
                   @Edad = 23,
                   @Sexo = 'M',
                   @RFC = '1234567890RFC',
                   @Correo = 'e.torressirangua@ugto.mx',
                   @Telefono = '5555555555',
                   @CURP = 'AAAA010101HGTAAA1',
                   @NUA = '792316',
                   @idCarrera = 1,
                   @Semestre = '10',
                   @Promedio = '9.4' 
go